In [297]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [298]:
#Predict whether a sentence comes from Alice in Wonderland or Persuasion.
#Removing the title. Match all text between square brackets and replace with empty string.


#Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

persuasion = re.sub(r'Chapter \d+', '', persuasion) #d+ is to remove the digits behind
alice = re.sub(r'CHAPTER .*', '', alice) #* is to remove everything behind.
#Without r, you'd have to type each backslash twice in order to pass it to re.sub.
#eg r'\]\n' is the same as '\\'\]\n' 


In [299]:
print(alice[0:100])

[Alice's Adventures in Wonderland by Lewis Carroll 1865]



Alice was beginning to get very tired of


In [300]:
def text_cleaner(text):
    text = re.sub(r'--', '', text)
    text = re.sub('[\[].*?![\]]', "", text)
    text = ' '.join(text.split())
    return text

alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [301]:
#Parse the cleaned novels.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [302]:
#Group into sentences
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, 'Austen'] for sent in persuasion_doc.sents]

# Cut Caesar down to the same length as Alice.
persuasion_sents = persuasion_sents[0:len(alice_sents)]

#Combine the sentences from the two novels into one df.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"([, Alice, 's, Adventures, in, Wonderland, by,...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [303]:
sentences['sentence_length']=sentences[0].str.len()
print(sentences.head())

                                                   0        1  sentence_length
0  ([, Alice, 's, Adventures, in, Wonderland, by,...  Carroll               78
1  (So, she, was, considering, in, her, own, mind...  Carroll               63
2  (There, was, nothing, so, VERY, remarkable, in...  Carroll               33
3                                      (Oh, dear, !)  Carroll                3
4                         (I, shall, be, late, !, ')  Carroll                6


In [304]:
# Utility function to create a list of the 1000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(1000)]
#item 0 is the word, item 1 is the count.

In [305]:
#Create a df with features for each word in our common word set.
#Each value is the count of the times the word appears.
#BOW is bag of words

def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df.loc[:, 'punctuation_length'] = 0
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    #loc you use the column name, iloc you use index (column number)
    #: means all rows. columns within common_words would be 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        puncts = [token for token in sentence if (token.is_punct)]
        df.loc[i,'punctuation_length'] += len(puncts)
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
    return df

In [306]:
#Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

#Combine bags to create a set of unique words.
#Set takes out duplicates
common_words = set(alicewords + persuasionwords)

In [307]:
#Creating the df with features.
word_counts = bow_features(sentences,common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000


,twist,week,till,small,differently,warm,strength,persuasion,railway,in,...,m,rooke,him,folly,purpose,edwin,completely,text_sentence,punctuation_length,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"([, Alice, 's, Adventures, in, Wonderland, by,...",12,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",7,Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",4,Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",1,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",2,Carroll


In [308]:
#Concat the number of words in that sentence into the df.
word_counts = pd.concat([word_counts, sentences['sentence_length']], axis=1)

In [309]:
print(word_counts.head())

   twist  week  till  small  differently  warm  strength  persuasion  railway  \
0      0     0     0      0            0     0         0           0        0   
1      0     0     0      0            0     0         0           0        0   
2      0     0     0      0            0     0         0           0        0   
3      0     0     0      0            0     0         0           0        0   
4      0     0     0      0            0     0         0           0        0   

   in       ...         rooke  him  folly  purpose  edwin  completely  \
0   0       ...             0    0      0        0      0           0   
1   0       ...             0    0      0        0      0           0   
2   0       ...             0    0      0        0      0           0   
3   0       ...             0    0      0        0      0           0   
4   0       ...             0    0      0        0      0           0   

                                       text_sentence  punctuation_length  

In [310]:
#Include column with number of words in the previous sentence.
#Create new column. Set to NULL
word_counts['previous_length'] = word_counts['sentence_length']
word_counts['previous_length'] = None

#df.shape returns two numbers, for example (10,10). The first number is the row count
for i in range(1,word_counts.shape[0]):
    word_counts.loc[i,'previous_length'] = word_counts.loc[i-1,'sentence_length']

In [311]:
#Set first sentence of Austen to NULL
first_sentence = min(word_counts.index[word_counts['text_source'] == 'Austen'].tolist())
word_counts.loc[first_sentence, 'previous_length'] = None

In [312]:
#Include column with number of words in the next sentence.
#Create new column. Set to NULL
word_counts['next_length'] = word_counts['sentence_length']
word_counts['next_length'] = None

In [313]:
for i in range(0,word_counts.shape[-1]):
    word_counts.loc[i,'next_length'] = word_counts.loc[i+1,'sentence_length']

In [314]:
#Set last sentence of Carroll to NULL
last_sentence = max(word_counts.index[word_counts['text_source'] == 'Carroll'].tolist())
word_counts.loc[last_sentence, 'next_length'] = None

In [315]:
print(word_counts.head())

   twist  week  till  small  differently  warm  strength  persuasion  railway  \
0      0     0     0      0            0     0         0           0        0   
1      0     0     0      0            0     0         0           0        0   
2      0     0     0      0            0     0         0           0        0   
3      0     0     0      0            0     0         0           0        0   
4      0     0     0      0            0     0         0           0        0   

   in     ...       folly  purpose  edwin  completely  \
0   0     ...           0        0      0           0   
1   0     ...           0        0      0           0   
2   0     ...           0        0      0           0   
3   0     ...           0        0      0           0   
4   0     ...           0        0      0           0   

                                       text_sentence  punctuation_length  \
0  ([, Alice, 's, Adventures, in, Wonderland, by,...                  12   
1  (So, she, was, 

In [176]:
#word_counts['text_sentence'] = word_counts['text_sentence'].astype(str)

In [316]:
word_counts['previous_length'] = word_counts['previous_length'].fillna(word_counts['previous_length'].mean())
word_counts['next_length'] = word_counts['next_length'].fillna(word_counts['next_length'].mean())

In [317]:
print(word_counts.head())

   twist  week  till  small  differently  warm  strength  persuasion  railway  \
0      0     0     0      0            0     0         0           0        0   
1      0     0     0      0            0     0         0           0        0   
2      0     0     0      0            0     0         0           0        0   
3      0     0     0      0            0     0         0           0        0   
4      0     0     0      0            0     0         0           0        0   

   in     ...       folly  purpose  edwin  completely  \
0   0     ...           0        0      0           0   
1   0     ...           0        0      0           0   
2   0     ...           0        0      0           0   
3   0     ...           0        0      0           0   
4   0     ...           0        0      0           0   

                                       text_sentence  punctuation_length  \
0  ([, Alice, 's, Adventures, in, Wonderland, by,...                  12   
1  (So, she, was, 

In [318]:
#Trying random forest
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'],1))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score', rfc.score(X_train, y_train))
print('\nTest set score', rfc.score(X_test, y_test))

Training set score 0.997986914947

Test set score 0.96679245283


In [319]:
from sklearn.model_selection import cross_val_score
cross_val_score(rfc, X, Y, cv=5)

array([ 0.97289157,  0.9652568 ,  0.97280967,  0.98338369,  0.90936556])

In [181]:
#Overfitting is usually a problem when using bag of words. 

In [320]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score', lr.score(X_train, y_train))
print('\nTest set score', lr.score(X_test, y_test))

(1987, 1567) (1987,)
Training set score 0.968797181681

Test set score 0.896603773585


In [321]:
from sklearn.model_selection import cross_val_score
cross_val_score(lr, X, Y, cv=5)

array([ 0.86746988,  0.86253776,  0.8836858 ,  0.89879154,  0.88217523])

In [322]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)
print('Training set score', clf.score(X_train, y_train))
print('\nTest set score', clf.score(X_test, y_test))

Training set score 0.989934574736

Test set score 0.979622641509


In [323]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf, X, Y, cv=5)

array([ 0.9939759 ,  1.        ,  1.        ,  1.        ,  0.86858006])

In [324]:
from sklearn.svm import SVR
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
svc = svm.SVC()
parameters = {'kernel':('linear', 'rbf')}
clf = GridSearchCV(svc, parameters)

train = clf.fit(X_train,y_train)
print('Training set score', clf.score(X_train, y_train))
print('\nTest set score', clf.score(X_test, y_test))

Training set score 0.980372420735

Test set score 0.887547169811


In [325]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf, X, Y, cv=5)

array([ 0.85090361,  0.84441088,  0.85951662,  0.88821752,  0.8776435 ])

Gradient Boosting Classifier achieved the best results for this model. Test score was 9.979! 

In [188]:
#Determine if the model is able to process Austen's Emma (instead of Persuasion) from Carroll with the 
#same amount of accuracy.

In [326]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

[Emma by Jane Austen 1816] Emma Woodhouse, handsome, clever, and rich, with a comfortable home and h


In [327]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [328]:
#Group into sentences
persuasion_sents = [[sent, 'Austen'] for sent in persuasion_doc.sents]
emma_sents = [[sent, 'Austen'] for sent in emma_doc.sents]

#Cut down Emma to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [329]:
#Build BOW df for Emma word counts. Use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_sentences['sentence_length'] = emma_sentences[0].str.len()
emma_bow = bow_features(emma_sentences, common_words)
print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [330]:
emma_bow['sentence_length'] = emma_sentences[0].str.len()

In [331]:
print(emma_bow.head())

   twist  week  till  small  differently  warm  strength  persuasion  railway  \
0      0     0     0      0            0     0         0           0        0   
1      0     0     0      0            0     0         0           0        0   
2      0     0     0      0            0     0         0           0        0   
3      0     0     0      0            0     0         0           0        0   
4      0     0     0      0            0     0         0           0        0   

   in       ...         rooke  him  folly  purpose  edwin  completely  \
0   0       ...             0    0      0        0      0           0   
1   0       ...             0    0      0        0      0           0   
2   0       ...             0    0      0        0      0           0   
3   0       ...             0    0      0        0      0           0   
4   0       ...             0    0      0        0      0           0   

                                       text_sentence  punctuation_length  

In [332]:
#Include column with number of words in the previous sentence.
#Create new column. Set to NULL
emma_bow['previous_length'] = emma_bow['sentence_length']
emma_bow['previous_length'] = None

#df.shape returns two numbers, for example (10,10). The first number is the row count
for i in range(1,emma_bow.shape[0]):
    emma_bow.loc[i,'previous_length'] = emma_bow.loc[i-1,'sentence_length']

In [333]:
print(emma_bow.head())

   twist  week  till  small  differently  warm  strength  persuasion  railway  \
0      0     0     0      0            0     0         0           0        0   
1      0     0     0      0            0     0         0           0        0   
2      0     0     0      0            0     0         0           0        0   
3      0     0     0      0            0     0         0           0        0   
4      0     0     0      0            0     0         0           0        0   

   in       ...         him  folly  purpose  edwin  completely  \
0   0       ...           0      0        0      0           0   
1   0       ...           0      0        0      0           0   
2   0       ...           0      0        0      0           0   
3   0       ...           0      0        0      0           0   
4   0       ...           0      0        0      0           0   

                                       text_sentence  punctuation_length  \
0  ([, Emma, by, Jane, Austen, 1816, ], 

In [237]:
#Set first sentence of Austen to NULL
#first_sentence = min(emma_bow.index[word_counts['text_source'] == 'Austen'].tolist())
#emma_bow.loc[first_sentence, 'previous_length'] = None

In [334]:
#Include column with number of words in the next sentence.
#Create new column. Set to NULL
emma_bow['next_length'] = emma_bow['sentence_length']
#emma_bow['next_length'] = None

In [335]:
for i in range(0,emma_bow.shape[-1]):
    emma_bow.loc[i,'next_length'] = emma_bow.loc[i+1,'sentence_length']

In [336]:
#Set last sentence of Austen to NULL
#last_sentence = max(emma_bow.index[emma_bow['text_source'] == 'Austen'].tolist())
#emma_bow.loc[last_sentence, 'next_length'] = None

emma_bow['previous_length'] = emma_bow['previous_length'].fillna(emma_bow['previous_length'].mean())
#emma_bow['next_length'] = emma_bow['next_length'].fillna(emma_bow['next_length'].mean())

In [337]:
print(emma_bow.head())

   twist  week  till  small  differently  warm  strength  persuasion  railway  \
0      0     0     0      0            0     0         0           0        0   
1      0     0     0      0            0     0         0           0        0   
2      0     0     0      0            0     0         0           0        0   
3      0     0     0      0            0     0         0           0        0   
4      0     0     0      0            0     0         0           0        0   

   in     ...       folly  purpose  edwin  completely  \
0   0     ...           0        0      0           0   
1   0     ...           0        0      0           0   
2   0     ...           0        0      0           0   
3   0     ...           0        0      0           0   
4   0     ...           0        0      0           0   

                                       text_sentence  punctuation_length  \
0  ([, Emma, by, Jane, Austen, 1816, ], Emma, Woo...                  10   
1  (She, was, the,

In [339]:
#Using logistic regression to model.
# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.685213414634


col_0,Austen,Carroll
row_0,,
Austen,1329,327
Carroll,499,469
